In [1]:
import csv
import math
import os
import subprocess
import sys
from collections import deque

from astropy.io import ascii
from astropy.coordinates import SkyCoord, Angle
from astropy.io import fits, votable
from astropy.wcs import WCS
import numpy as np
import numpy.core.records as rec
from astropy import units as u
from astropy.table import QTable, Table, Column


## Prepare source cutout run

This notebook sets up the parameters for bulk source cutouts.

It will create the following files which can be used as input by askap_cutout_daemon.py to produce fits cutouts around the target sources:
* sb10941_srcs_image_params.vot
* targets_10941.csv

It takes the beam listings as input, e.g. beam_listing_SB10941.csv as well as the Selavy continuum source catalogue for the field. The two output files contain the selected source details (name, location) and the beams that are close to the source and should be used to produce the cutout cubelet. The names of the output files are based on the input ASKAP sbid.

Note this code assumes the use of a 36 beam pattern with 3 interleaves, so a total of 108 beams. It also assumes only one set of beams. If multiple observations are to be included, then the measurements sets for all observations must be concatenated so there is a single measurement set per beam.

### Cutout process

askap_cutout_daemon.py should be run on the login node of a cluster (e.g. avatar). It will manage a series of jobs, one for each source, keeping a defined number running at a time. Each job will use CASA to produce a cubelet around the source based on the beams that are close enough to the source to be provide data for the source.

Each job runs make_askap_abs_cutout.sh to start CASA to run the sub_cube_abs.py script. This has the CASA commands to produce the cubelet. It will create a cubelet covering just the immediate region f the source, it will exclude short baselines, to limit the large scale emission included, and will not do any cleaning.

### Input data

The beam listings are produced by running a list_beams script (e.g. list_beams_10941.py) in CASA which extracts the beam positions from each beam measurement set. This assumes the measurement sets have been recentred from their common position output by ASKAPSoft.

The catalogue is the continuum catalogue produced with Selavy as part of the ASKAPSoft pipeline processing.

### Output Data

In [2]:
# Parameters for the run
sbid = 10941
catalogue = 'AS108_Continuum_Component_Catalogue_10941_1559.votable'
beam_listing = 'beam_listing_SB{}.csv'.format(sbid)
flux_peak_min = 15 # mJy/beam

In [3]:
# Read and filter catalogue
src_votable = votable.parse(catalogue, pedantic=False)
table = src_votable.get_first_table().to_table()
targets = table[table['flux_peak']>flux_peak_min]
targets

id,catalogue_id,first_sbid,other_sbids,project_id,island_id,component_id,component_name,ra_hms_cont,dec_dms_cont,ra_deg_cont,dec_deg_cont,ra_err,dec_err,freq,flux_peak,flux_peak_err,flux_int,flux_int_err,maj_axis,min_axis,pos_ang,maj_axis_err,min_axis_err,pos_ang_err,maj_axis_deconv,min_axis_deconv,maj_axis_deconv_err,pos_ang_deconv,min_axis_deconv_err,pos_ang_deconv_err,chi_squared_fit,rms_fit_gauss,spectral_index,spectral_index_err,spectral_curvature,spectral_curvature_err,rms_image,has_siblings,fit_is_estimate,spectral_index_from_tt,flag_c4,comment,quality_level,released_date
,,,,,,,,,,deg,deg,arcsec,arcsec,MHz,mJy/beam,mJy/beam,mJy,mJy,arcsec,arcsec,deg,arcsec,arcsec,deg,arcsec,arcsec,arcsec,deg,arcsec,deg,,mJy/beam,,,,,mJy/beam,,,,,,,
int64,int64,int32,bytes500,int64,bytes255,bytes256,bytes32,bytes16,bytes16,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,bytes1000,bytes15,bytes24
4750881,1559,10941,,19,SB10941_island_334,SB10941_component_334a,J004725-712727,00:47:25.4,-71:27:27,11.85596,-71.457707,0.01,0.01,1419.5,15.015,0.059,16.122,0.09,9.56,8.18,133.63,0.04,0.01,1.02,2.58,2.05,0.66,-67.75,1.1,3.61,92.861,1064.165,-99.0,0.0,-99.0,0.0,0.124,0,0,--,0,,NOT_VALIDATED,
4750878,1559,10941,,19,SB10941_island_332,SB10941_component_332a,J010653-693928,01:06:53.4,-69:39:28,16.722322,-69.658048,0.02,0.02,1419.5,15.013,0.076,16.096,0.121,9.66,8.08,139.79,0.05,0.01,1.21,2.96,1.56,0.24,-25.09,0.99,2.21,13.848,537.112,-99.0,0.0,-99.0,0.0,0.297,0,0,--,0,,NOT_VALIDATED,
4750873,1559,10941,,19,SB10941_island_330,SB10941_component_330a,J010234-695828,01:02:34.0,-69:58:28,15.64157,-69.974491,0.02,0.02,1419.5,15.157,0.097,16.129,0.15,9.59,8.08,134.54,0.06,0.02,1.54,2.64,1.7,0.61,-52.23,1.62,4.38,60.766,952.342,-99.0,0.0,-99.0,0.0,0.224,0,0,--,0,,NOT_VALIDATED,
4750872,1559,10941,,19,SB10941_island_329,SB10941_component_329a,J014039-725101,01:40:39.1,-72:51:01,25.163098,-72.850402,0.01,0.01,1419.5,15.037,0.055,17.712,0.09,9.47,9.06,120.79,0.03,0.01,3.4,4.52,1.96,0.01,52.25,0.11,0.33,41.36,714.573,-99.0,0.0,-99.0,0.0,0.18,0,0,--,0,,NOT_VALIDATED,
4750871,1559,10941,,19,SB10941_island_328,SB10941_component_328a,J002620-743741,00:26:20.8,-74:37:41,6.586833,-74.628167,0.03,0.03,1419.5,15.201,0.105,17.953,0.172,10.14,8.48,135.65,0.07,0.02,1.57,4.21,3.1,0.15,-44.63,0.32,2.97,166.888,1354.23,-99.0,0.0,-99.0,0.0,0.183,0,0,--,0,,NOT_VALIDATED,
4750870,1559,10941,,19,SB10941_island_327,SB10941_component_327a,J001807-723924,00:18:07.9,-72:39:24,4.532889,-72.656755,0.02,0.02,1419.5,15.258,0.087,15.931,0.136,9.74,7.81,141.72,0.06,0.01,1.08,3.24,0.0,0.11,-27.37,0.0,1.57,7.644,391.01,-99.0,0.0,-99.0,0.0,0.464,0,0,--,0,,NOT_VALIDATED,
4750866,1559,10941,,19,SB10941_island_325,SB10941_component_325a,J013350-700419,01:33:50.6,-70:04:19,23.460916,-70.072185,0.02,0.02,1419.5,15.402,0.075,16.521,0.116,9.79,7.98,145.88,0.05,0.01,0.98,3.58,0.0,0.07,-16.27,0.0,1.12,50.991,865.951,-99.0,0.0,-99.0,0.0,0.192,0,0,--,0,,NOT_VALIDATED,
4750860,1559,10941,,19,SB10941_island_322,SB10941_component_322a,J002430-721225,00:24:30.3,-72:12:25,6.126414,-72.207051,0.03,0.03,1419.5,15.292,0.12,16.691,0.187,9.62,8.27,131.58,0.08,0.02,2.15,2.92,2.2,0.67,-79.84,1.28,4.21,46.115,848.853,-99.0,0.0,-99.0,0.0,0.31,0,0,--,0,,NOT_VALIDATED,


In [4]:
def slice_strings(str_array,start,end):
    """
    Takes an array of strings and slices each string.
    str_array: The array of strings
    start: The start index, may be negative to indicate a reference from the end of the strings
    end: The end index, may be None when start is negative
    """
    if end is None:
        if start > 0:
            raise('end must be present when start is positive')
        b = str_array.view((str,1)).reshape(len(str_array),-1)[:,start:]
        return np.frombuffer(b.tostring(),dtype=(str,start*-1))
    
    b = str_array.view((str,1)).reshape(len(str_array),-1)[:,start:end]
    return np.frombuffer(b.tostring(),dtype=(str,end-start))

In [5]:
# Build the list of beam locations
beams = ascii.read(beam_listing, format='no_header', guess=False, delimiter=',')
names = ('col1','col2', 'col3', 'col4')
new_names = ('filename','interleave', 'ra_rad', 'dec_rad')
beams.rename_columns(names, new_names)

beam_id = slice_strings(beams['filename'], -8, -6)
interleave_id = slice_strings(beams['interleave'], -1, None)# beams['interleave'][:][-1:]
file_interleave = slice_strings(beams['filename'], -14, -13)

ids = np.stack((beam_id, interleave_id), axis=-1)
unique_ids, unique_idx = np.unique(ids, axis=0, return_index=True)

beams['beam_id'] = beam_id
beams['interleave'] = interleave_id
beams['file_interleave'] = file_interleave
unique_beams = beams[beams['interleave'] == beams['file_interleave']]
print (unique_beams)

u_beam_locs = SkyCoord(ra=unique_beams['ra_rad']*u.rad, dec=unique_beams['dec_rad']*u.rad, frame='icrs')
u_beam_locs


                                                              filename                                                               ...
------------------------------------------------------------------------------------------------------------------------------------ ...
/avatar/nipingel/ASKAP/SMC/data/pilot_obs/ms_data/10941/GASKAP_M344-11B_T0-0A/scienceData_SB10941_GASKAP_M344-11B_T0-0A.beam00_SL.ms ...
/avatar/nipingel/ASKAP/SMC/data/pilot_obs/ms_data/10941/GASKAP_M344-11B_T0-0A/scienceData_SB10941_GASKAP_M344-11B_T0-0A.beam01_SL.ms ...
/avatar/nipingel/ASKAP/SMC/data/pilot_obs/ms_data/10941/GASKAP_M344-11B_T0-0A/scienceData_SB10941_GASKAP_M344-11B_T0-0A.beam02_SL.ms ...
/avatar/nipingel/ASKAP/SMC/data/pilot_obs/ms_data/10941/GASKAP_M344-11B_T0-0A/scienceData_SB10941_GASKAP_M344-11B_T0-0A.beam03_SL.ms ...
/avatar/nipingel/ASKAP/SMC/data/pilot_obs/ms_data/10941/GASKAP_M344-11B_T0-0A/scienceData_SB10941_GASKAP_M344-11B_T0-0A.beam04_SL.ms ...
/avatar/nipingel/ASKAP/SMC/data/pilot_obs

<SkyCoord (ICRS): (ra, dec) in deg
    [( 5.81673852, -74.43078655), ( 9.17112006, -74.4913961 ),
     (12.53893812, -74.50160532), (15.89980626, -74.46136404),
     (19.23359811, -74.37106166), (22.52119108, -74.23150921),
     ( 7.68056192, -73.68942536), (10.88766415, -73.72318603),
     (14.09748484, -73.70883677), (17.29249828, -73.64652742),
     (20.45568081, -73.53677888), (23.57106596, -73.38046122),
     ( 6.32507933, -72.8781978 ), ( 9.3832877 , -72.9333783 ),
     (12.45137739, -72.94264711), (15.51412796, -72.90597428),
     (18.55648344, -72.82365376), (21.56401323, -72.69629225),
     ( 8.00770647, -72.13351853), (10.9432524 , -72.16440342),
     (13.88072025, -72.15126557), (16.80684739, -72.0942231 ),
     (19.70869125, -71.99367156), (22.5739836 , -71.85026997),
     ( 6.7453963 , -71.32468164), ( 9.55723909, -71.37542063),
     (12.3765317 , -71.38394768), (15.19161644, -71.35024604),
     (17.99094406, -71.27454339), (20.76337144, -71.15730463),
     ( 8.27953719, -

In [6]:
def get_beams_near_src(target_loc, beam_locs, beams, max_sep = 0.8*(1*u.deg)):
    """
    Find the beams within a certain angular distance of the target location.
    
    Parameters
    ----------
    target_loc: SkyCoord
        The sky location of the target.
    beam_locs: SkyCoord[]
        The locations of each beam.
    beams: str[]
        Array of beam names
    max_sep: dimension
        Maximum distance a beam centre can be from the target to be included (optional).
    
    Returns
    -------
    List of beam names within the requested distance and a list of their distances from the target.
    """
    beam_sep = beam_locs.separation(target_loc)
    beams_covering_target = beam_sep < max_sep
    src_beams = beams[beams_covering_target]
    src_beam_sep = beam_sep[beams_covering_target]
    return src_beams, src_beam_sep



In [7]:
comp_names = []
comp_ra = []
comp_dec = []
included_beam_nums = []
included_beam_interleaves = []
included_beam_ids = []
included_beam_sep = []

for tgt in targets:
    target_loc = SkyCoord(ra=tgt['ra_deg_cont']*u.degree, dec=tgt['dec_deg_cont']*u.degree, frame='icrs')
    src_beams, src_beam_sep = get_beams_near_src(target_loc, u_beam_locs, unique_beams)
    for i in range(len(src_beams)):
        comp_names.append(tgt['component_name'])
        comp_ra.append(tgt['ra_deg_cont'])
        comp_dec.append(tgt['dec_deg_cont'])
        included_beam_nums.append(src_beams['beam_id'].data[i])
        included_beam_interleaves.append(src_beams['interleave'].data[i])
        included_beam_ids.append(src_beams['beam_id'].data[i]+src_beams['interleave'].data[i])
        included_beam_sep.append(src_beam_sep.to(u.deg).value[i])

image_params = Table()
image_params['component_name'] = comp_names
image_params['comp_ra'] = comp_ra
image_params['comp_dec'] = comp_dec
image_params['beam_nums'] = included_beam_nums
image_params['beam_interleaves'] = included_beam_interleaves
image_params['beam_ids'] = included_beam_ids
image_params['beam_sep'] = included_beam_sep

image_params_vot = votable.from_table(image_params)
filename = "sb{}_srcs_image_params.vot".format(sbid)
votable.writeto(image_params_vot, filename)

print ("Produced VO table file {} with the following contents:".format(filename))

image_params_vot.get_first_table()


Produced VO table file sb10941_srcs_image_params.vot with the following contents:


<Table masked=True length=2143>
component_name  comp_ra   comp_dec  ... beam_ids       beam_sep     
    str14       float64   float64   ...   str3         float64      
-------------- --------- ---------- ... -------- -------------------
J004725-712727  11.85596 -71.457707 ...      19A  0.7619438663909153
J004725-712727  11.85596 -71.457707 ...      25A   0.737129937158118
J004725-712727  11.85596 -71.457707 ...      26A 0.18152219985457238
J004725-712727  11.85596 -71.457707 ...      19B 0.47752368805038975
J004725-712727  11.85596 -71.457707 ...      25B 0.43670210167811485
J004725-712727  11.85596 -71.457707 ...      26B  0.7001350150557226
J004725-712727  11.85596 -71.457707 ...      25C 0.33997446013179344
J004725-712727  11.85596 -71.457707 ...      26C   0.644470560271725
J004725-712727  11.85596 -71.457707 ...      31C  0.6149609046710358
J010653-693928 16.722322 -69.658048 ...      33B  0.6658505090257966
           ...       ...        ... ...      ...                 ...
J0

In [8]:
print ("Count of the number of sources using each beam")
ar = np.array(included_beam_ids)
for i in range(36):
    for interleave in ('A', 'B', 'C'):
        key = '{:02d}{}'.format(i, interleave)
        count = len(ar[ar==key])
        print (key, count)
print ('average', len(ar)/(36*3))

Count of the number of sources using each beam
00A 27
00B 31
00C 23
01A 20
01B 12
01C 8
02A 11
02B 11
02C 11
03A 13
03B 11
03C 12
04A 12
04B 19
04C 11
05A 16
05B 17
05C 13
06A 25
06B 17
06C 23
07A 15
07B 18
07C 11
08A 10
08B 18
08C 11
09A 22
09B 24
09C 22
10A 25
10B 21
10C 19
11A 20
11B 24
11C 20
12A 12
12B 13
12C 20
13A 20
13B 18
13C 21
14A 21
14B 20
14C 17
15A 27
15B 19
15C 24
16A 23
16B 13
16C 22
17A 12
17B 13
17C 22
18A 14
18B 16
18C 18
19A 16
19B 25
19C 24
20A 28
20B 25
20C 29
21A 21
21B 23
21C 21
22A 14
22B 17
22C 13
23A 12
23B 13
23C 16
24A 22
24B 27
24C 15
25A 19
25B 23
25C 16
26A 23
26B 24
26C 22
27A 26
27B 17
27C 19
28A 16
28B 20
28C 16
29A 23
29B 28
29C 21
30A 27
30B 30
30C 27
31A 27
31B 29
31C 30
32A 22
32B 23
32C 20
33A 22
33B 23
33C 18
34A 26
34B 33
34C 26
35A 29
35B 26
35C 23
average 19.84259259259259


In [9]:
ar = np.array(included_beam_ids)
print ('Count of total beam usage={}\nNumber using beam 02A={}'.format(len(ar), len(ar[ar=='02A'])))

Count of total beam usage=2143
Number using beam 02A=11


In [10]:
print ('Example source and included beams')
tgt = targets[100]
target_loc = SkyCoord(ra=tgt['ra_deg_cont']*u.degree, dec=tgt['dec_deg_cont']*u.degree, frame='icrs')
src_beams, src_beam_sep = get_beams_near_src(target_loc, u_beam_locs, unique_beams)
print(tgt['component_name'])
print(tgt['ra_deg_cont'])
print(tgt['dec_deg_cont'])
print(src_beams['beam_id'].data)

Example source and included beams
J010401-720206
16.004519
-72.035218
['20' '21' '27' '15' '20' '21' '20' '21' '26' '27']


In [11]:
# Create the csv file of targets and included beams
csv_filename = 'targets_{}.csv'.format(sbid)
with open(csv_filename, 'w') as csvfile:
    tgtwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    tgtwriter.writerow(['index', 'component_name', 'ra', 'dec', 'beams'])
    i = 1;
    for tgt in targets:
        comp_name = tgt['component_name']
        row = []
        row.append(str(i))
        row.append(comp_name)
        row.append(tgt['ra_deg_cont'])
        row.append(tgt['dec_deg_cont'])
        for tgt_beam in image_params:
            if comp_name == tgt_beam['component_name']:
                row.append(tgt_beam['beam_ids'])
        tgtwriter.writerow(row)
        i+= 1

print ('Produced csv file {} with {} targets'.format(csv_filename, i))

Produced csv file targets_10941.csv with 374 targets
